In [1]:
import os
import tensorflow as tf
import numpy as np
import tvm
import tvm.relay as relay
import tvm.contrib.graph_runtime as runtime
from tvm.relay.expr_functor import ExprMutator
import riptide.models
from riptide.get_models import get_model
from riptide.binary.binary_layers import Config, DQuantize, XQuantize

In [2]:
os.environ["CUDA_VISIBLE_DEVICES"] = ''

In [3]:
config = Config(actQ=DQuantize, weightQ=XQuantize, bits=1, use_act=False, use_bn=False, use_maxpool=True)

In [4]:
#with config:
#    model = get_model('vggnet')
model = riptide.models.vggnet_normal.vggnet()
#model = tf.keras.models.Sequential()
#model.add(tf.keras.layers.Conv2D(filters=96, strides=2, padding='same', kernel_size=7, use_bias=False, data_format='channels_last'))
#model.add(tf.keras.layers.BatchNormalization(center=False, scale=False))

In [5]:
test_input = tf.keras.Input(shape=[224, 224, 3], batch_size=1, dtype='float32')
output = model(test_input)

In [6]:
func, params = relay.frontend.from_keras(model, shape={'input_1': [1, 224, 224, 3]})

In [7]:
with relay.build_config(opt_level=3):
    #graph, lib, params = relay.build(func, 'llvm', params=params)
    graph, lib, params = relay.build_module.build(func, target='llvm', params=params)
    #out = intrp.evaluate(func)(np.random.uniform(size=(1, 3, 28, 28)))

W0613 00:00:05.150222 140685900666688 dispatcher.py:381] Cannot find config for target=llvm, workload=('conv2d', (1, 3, 229, 229, 'float32'), (96, 3, 7, 7, 'float32'), (2, 2), (0, 0), (1, 1), 'NCHW', 'float32'). A fallback configuration is used, which may bring great performance regression.
W0613 00:00:05.250350 140685900666688 dispatcher.py:381] Cannot find config for target=llvm, workload=('bitserial_dense', (1, 4096, 'int16'), (1000, 1, 128, 'uint32'), 1, 1, 'uint32', 'int16', 1). A fallback configuration is used, which may bring great performance regression.
W0613 00:00:05.272911 140685900666688 dispatcher.py:381] Cannot find config for target=llvm, workload=('bitserial_dense', (1, 4096, 'int16'), (4096, 1, 128, 'uint32'), 1, 1, 'uint32', 'int16', 1). A fallback configuration is used, which may bring great performance regression.
W0613 00:00:05.295073 140685900666688 dispatcher.py:381] Cannot find config for target=llvm, workload=('bitserial_dense', (1, 25088, 'int16'), (4096, 1, 7

In [8]:
module = runtime.create(graph, lib, tvm.context('llvm', 0))
module.set_input(**params)

In [9]:
module.set_input('input_1', np.random.uniform(size=(1, 3, 224, 224)))
module.run()
print(module.get_output(0).shape)

(1, 1000)


In [10]:
# evaluate
ctx = tvm.cpu()
print("Evaluate inference time cost...")
ftimer = module.module.time_evaluator("run", ctx, number=1, repeat=100)
prof_res = np.array(ftimer().results) * 1000  # convert to millisecond
print("Mean inference time (std dev): %.2f ms (%.2f ms)" %
      (np.mean(prof_res), np.std(prof_res)))

Evaluate inference time cost...
Mean inference time (std dev): 211.00 ms (12.63 ms)
